In [1]:
df = spark.read.format("parquet").load(
    "Files/silver/taxi/fact_taxi_trips"
)



StatementMeta(, c9344fa4-f20a-4893-bd8e-f825e225022d, 3, Finished, Available, Finished)

In [7]:
df.printSchema()


StatementMeta(, 10fc343e-5bb3-4bc9-b328-1c77c9865f39, 9, Finished, Available, Finished)

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_ts: timestamp (nullable = true)
 |-- dropoff_ts: timestamp (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- pu_zone_id: integer (nullable = true)
 |-- do_zone_id: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_date: date (nullable = true)



In [2]:
from pyspark.sql.functions import (
    count, sum, avg
)

gold_daily_zone = (
    df.groupBy(
        "pickup_date",
        "pu_zone_id"
    )
    .agg(
        count("*").alias("trips_cnt"),
        sum("fare_amount").alias("total_fare"),
        avg("fare_amount").alias("avg_fare"),
        sum("total_amount").alias("total_revenue"),
        sum("tip_amount").alias("total_tips"),
        avg("trip_distance").alias("avg_distance"),
        avg("passenger_count").alias("avg_passengers")
    )
)


StatementMeta(, c9344fa4-f20a-4893-bd8e-f825e225022d, 4, Finished, Available, Finished)

In [5]:
gold_daily_zone.write \
    .mode("overwrite") \
    .format("delta") \
    .save("Files/gold/taxi/gold_trips_daily_pickup_zone")


StatementMeta(, 10fc343e-5bb3-4bc9-b328-1c77c9865f39, 7, Finished, Available, Finished)

In [5]:
gold_daily_zone.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("dbo.gold_taxi_trips_daily")


StatementMeta(, c9344fa4-f20a-4893-bd8e-f825e225022d, 7, Finished, Available, Finished)

In [6]:
spark.sql("""
SELECT *
FROM dbo.gold_taxi_trips_daily;
""").show()

StatementMeta(, c9344fa4-f20a-4893-bd8e-f825e225022d, 8, Finished, Available, Finished)

+-----------+----------+---------+------------------+------------------+------------------+------------------+------------------+------------------+
|pickup_date|pu_zone_id|trips_cnt|        total_fare|          avg_fare|     total_revenue|        total_tips|      avg_distance|    avg_passengers|
+-----------+----------+---------+------------------+------------------+------------------+------------------+------------------+------------------+
| 2025-05-15|       265|       32|3083.3999999999996| 96.35624999999999|           3443.87|160.60999999999999|         8.0865625|             1.625|
| 2025-05-08|        81|        6|             245.0|40.833333333333336|             272.0|               0.0|             11.25|               1.0|
| 2025-05-08|        32|        4|             149.0|             37.25|            157.94|               0.0|              8.55|               1.0|
| 2025-10-16|       261|      629| 17288.09999999999| 27.48505564387916| 24573.25999999998|3312.3099999999